In [35]:
#this is my vendor and ground truth data that is coming in 
#we will parse it out into individual elements

class Vendor_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in vendor event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]
        
class GroundTruth_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in ground truth event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]
        

In [184]:

class Vendor_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in vendor event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]
        
class GroundTruth_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in ground truth event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]


'''#import python;
#import aic;

#// spurious - event in vendor set that's not in the gold set
#// unmatched - event in gold set that's not in the vendor set

// For gold records with no events, calculate spurious outside of this call
// For gold records with no vendor records, calculate unmatched outside of this call
// Only make this call if there are gold records AND vendor records for the given id.
// Make this call passing in ALL gold records with the given ID, and ALL vendor records with the given ID.
// Make this call for each ID independently.

// ----- This version only computes the score for the event type field.

export dataset(aic.score.Layouts.L_score) do_computescores2(dataset(aic.score.Layouts.L_in) ds_gold, dataset(aic.score.Layouts.L_in) ds_vendor, utf8 vendor_code, utf8 runid, utf8 gt_cameo_id, utf8 gt_civilunrest_id) := EMBED(Python)
__author__ = "Wei Wang"
__email__ = "tskatom@vt.edu"'''

import sys
import os
from munkres import Munkres
import numpy as np
import re
import csv

import collections
import itertools

def reduce_vendordata (vendorlist):
    
    #print len(vendorlist)
    #print vendorlist

    while_counter = 0
    vendoractor1 = []
    vendoractor2 = []
    vendoractor1mentions = []
    vendoractor2mentions = []
    vendorfullgeonames = []
    vendorcameocodes = []
    vendorguid = []
    vendorid = []
    
    

    while while_counter < len(vendorlist):
    
        #get all of our values for scoring
    
        #print while_counter,dsvendor[while_counter][1]
        vendorid.append(vendorlist[while_counter][0])
        vendoractor1.append(vendorlist[while_counter][1])
        vendoractor2.append(vendorlist[while_counter][2])
        vendoractor1mentions.append(vendorlist[while_counter][3])
        vendoractor2mentions.append(vendorlist[while_counter][4])
        vendorfullgeonames.append(vendorlist[while_counter][5])
        vendorcameocodes.append(vendorlist[while_counter][6])
        vendorguid.append(vendorlist[while_counter][7])
        clusterid2 = vendorlist[while_counter][8]
        
        #print 'in flatten clusterid',clusterid2
        while_counter += 1
    

    #lets check our values
    #list(set(gt_guid))
    #filter(None, lstr)
    #we are removing all null entries and duplicates just to get to the bare minimums
    vendoractor1 = filter(None,list(set(vendoractor1)))
    vendoractor2 = filter(None,list(set(vendoractor2)))
    vendoractor1mentions = filter(None,list(set(vendoractor1mentions)))
    vendoractor2mentions = filter(None,list(set(vendoractor2mentions)))
    vendorfullgeonames = filter(None,list(set(vendorfullgeonames)))
    vendorcameocodes = filter(None,list(set(vendorcameocodes)))
    vendorguid = filter(None,list(set(vendorguid)))

    
    #print 'a1',vendoractor1
    #print 'a2',vendoractor2
    #print 'a1m',vendoractor1mentions
    #print 'a2m',vendoractor2mentions
    #print 'fgn',vendorfullgeonames
    #print 'cameo',vendorcameocodes
    #print 'guid',vendorguid
    #print 'id',vendorid
    #print clusterid
    
    return vendoractor1,vendoractor2,vendoractor1mentions,vendoractor2mentions,vendorfullgeonames,vendorcameocodes,vendorguid,vendorid,clusterid2
    #end of vendor data function

def reduce_gtdata (gtlist):
    
    #print len(gtlist)

    while_counter = 0
    gtactor1 = []
    gtactor2 = []
    gtactor1mentions = []
    gtactor2mentions = []
    gtfullgeonames = []
    gtcameocodes = []
    gtguid = []

    while while_counter < len(gtlist):
    
        #get all of our values for scoring
    
        #print while_counter,dsgt[while_counter][1]
    
        gtactor1.append(gtlist[while_counter][1])
        gtactor2.append(gtlist[while_counter][2])
        gtactor1mentions.append(gtlist[while_counter][3])
        gtactor2mentions.append(gtlist[while_counter][4])
        gtfullgeonames.append(gtlist[while_counter][5])
        gtcameocodes.append(gtlist[while_counter][6])
        gtguid.append(gtlist[while_counter][7])
    
        while_counter += 1
    

    #lets check our values
    #list(set(gt_guid))
    #filter(None, lstr)
    #we are removing all null entries and duplicates just to get to the bare minimums
    gtactor1 = filter(None,list(set(gtactor1)))
    gtactor2 = filter(None,list(set(gtactor2)))
    gtactor1mentions = filter(None,list(set(gtactor1mentions)))
    gtactor2mentions = filter(None,list(set(gtactor2mentions)))
    gtfullgeonames = filter(None,list(set(gtfullgeonames)))
    gtcameocodes = filter(None,list(set(gtcameocodes)))
    gtguid = filter(None,list(set(gtguid)))

    #print 'gt1',gtactor1
    #print 'gt2',gtactor2
    #print 'gt1m',gtactor1mentions
    #print 'gt2m',gtactor2mentions
    #print 'gtfgn',gtfullgeonames
    #print 'gtcameo',gtcameocodes
     
    return gtactor1,gtactor2,gtactor1mentions,gtactor2mentions,gtfullgeonames,gtcameocodes,gtguid
    #end of gt data function    
    
def compute_score(true_event, test_event):
    
	#print 'in compute_score check our gt values',true_event.actor1name
	#print 'in compute_score check our vendor values',test_event.actor1name
	# """
	# Evaluate the score of test event
	# parameters:
	#    true_event: json 
	#    test_event: json
	# """
	################
	# score actor1 #
	################
	# only evaluate the actor1name with true event using partial match
	actor1_score = 0.0
	test_actor1 = test_event.actor1name
	#print 'print vendor actor1',test_actor1    
	# if there are multiple sources, we match the actor1 with 
	# highest score. In Reciproal case, we compare the actor1/2 with 
	# the same set of Reciprocal texts.
	if true_event.actor1name:
		sources = true_event.actor1name.split(";")
	else:
		sources = []
	if true_event.actor1mention:
		new_sources = true_event.actor1mention.split(";")
		for new_source in new_sources:
			sources.append(new_source)

	act_score = 0.
	for source in sources:
		act_score = 3.0 if text_partial_match(source, test_actor1) else 0.
		if act_score > actor1_score:
			actor1_score = act_score
	#print 'actor1 score',act_score
	################
	# score actor2 #
	################
	actor2_score = 0.0
	test_actor2 = test_event.actor2name
	#print 'print vendor actor2',test_actor2
    
	if true_event.actor2name:
		targets = true_event.actor2name.split(";")
	else:
		targets = []
	if true_event.actor2mention:
		new_targets = true_event.actor2mention.split(";")
		for new_target in new_targets:
			targets.append(new_target)

	for target in targets:
		#print 'in targets',target    
		actor_score = 3.0 if text_partial_match(target, test_actor2) else 0
		#print 'actor2 score',actor_score
		if act_score > actor2_score:
			actor2_score = act_score

	##################
	# Score Location #
	##################
	# Match Location with partial match
	location_score = 0.0
	true_location = true_event.actiongeo_fullname.split(";")
	test_location = test_event.actiongeo_fullname.strip()
	#print 'print gt location',true_location    
	#print 'print vendor location',test_location
	if len(true_location) == 0:
		if test_location == "":
			location_score = 3.0
		else:
			location_score = 0.0
	else:
		for loc in true_location:
			tem_score = 3.0 if text_partial_match(loc, test_location) else 0.0
			if tem_score > location_score:
				location_score = tem_score
    
	################
	# score action #
	################
	# compute the action score with two level: root and subtype
	action_score = 0.0
	true_code = true_event.eventcode.split(" ")[0]
	test_code = test_event.eventcode.split(" ")[0]
    
	#print 'checking our event code values for gt -->',true_code
	#print 'checking our test code values for gt -->',test_code
    
	if test_code is None or len(test_code) < 2:
		#print 'in action score fail'        
		action_score = 0.0
	else:
		if true_code[:2] == test_code[:2]:
			#print 'checking action score',true_code[:2],test_code[:2],action_score
			action_score += 2.0
			if len(test_code) > 3 and test_code[2] == true_code[2]:
				action_score += 1.0
    
	###############
	# Date Score ##
	###############
	# since the Date field in the GSR is not normalized, we don't compute the
	# score here
	date_score = 0.0

	final_score = [actor1_score, actor2_score, location_score, action_score, date_score]

    
	fields = ["Actor1", "Actor2", "Location", "Action", "Date"]
	#print 'Final_Score -->',sum(final_score),dict(zip(fields, final_score))

	return sum(final_score), dict(zip(fields, final_score))

def text_partial_match(true_text, test_text):
	"""
	partial match the true_text and test_text based on following step
	1> transform the text to lowercase 
	2> split the text by space or comma and ignore ''
	3> if any pair of elements from two different text arraies equal to each
	other, return True else False
	"""
	true_text_splits = set([r for r in re.split(r" |,", true_text.lower()) if r != ''])
	test_text_splits = set([r for r in re.split(r" |,", test_text.lower()) if r != ''])
    
	if true_text_splits.intersection(test_text_splits):
		return True
	else:
		return False

def global_match(ground_truth, vendor_events):
	"""
	matching the true events with test events with global optimal score
	"""
	#print 'here'
	#print 'ground_truth',len(ground_truth)
	#print 'vendor',len(vendor_events)
    
#	if not true_events or not test_events:
#		return 0.0, [], [], []

	#this was original len check, but i think we need to check how many elements are in our inbound list, so changed it     
	#print 'here and length of true_events',len(true_events)    
	#print 'here is the length of our ground truth list-->',len(ground_truth)
	#print 'here is the length of our vendor list-->',len(vendor_events)
    
	gt_guid = []    
	vendor_guid = []
	vendor_score = 0.00
    
	#print vendor_events
    
	for records, true_event in enumerate(ground_truth):
		#if records == len(ground_truth): break        
		#extract guid
		
		#print records        
		gt_guid.append(ground_truth[records][7])

	#print 'before vendor guid check'       
	#print vendor_events
        
	for counter, records in enumerate(vendor_events):
		#extract guid
		vendor_guid.append(vendor_events[counter][7])
	#print 'all of my gt guids',gt_guid
	#print 'all of my vendor guids',vendor_guid
    
	#lets define our set of guids
	gtguids = list(set(gt_guid))

	#print 'here is my gt set', gtguids
    
	vendorguid = list(set(vendor_guid))
	#print 'here is my vendor set', vendorguid
    
	#let's check how many values are in our vendor set
	guids_to_process = []    
	for guids,groundtruth in enumerate(gtguids):
		#print 'checking',gt         
		if gtguids[guids] in vendorguid:
			guids_to_process.append(gtguids[guids])  
		#else:        
			#print 'no'   
			
	#print 'here are my guids_to_process',guids_to_process 

	#ok, so now lets iterate through out guids
	#print 'before guid count'  
	#score_matrix = []    
	for processing in guids_to_process:
       
		vendor_1 = filter(lambda vendor_events: vendor_events[7] in processing, vendor_events) 
		gt_1  = filter(lambda ground_truth: ground_truth[7] in processing, ground_truth)        
		#print 'vendors',vendor_1    
		#print 'gt',gt_1           
        
		score_matrix = np.zeros((len(gt_1),len(vendor_1)))  
		#print 'score matrix -->',score_matrix 
        
		#print 'score matrix -->',score_matrix    
    
		score_detail = {}
		for i, true_event in enumerate(gt_1):
		#if i%500==0:
				#print 'outer loop divisible by 500 counter is @',i
		#print 'loop counter',i
			for j, test_event in enumerate(vendor_1):
				true_events = GroundTruth_Data(gt_1,i)

			#if j%500==0:
				#print 'inner loop divisible by 500 counter is @',j            
    
			#print 'check true event value', true_events.actor1name
    
				test_events = Vendor_Data(vendor_1,j)    
			#print 'check vendor event value', test_events.actor1name
			#print 'loop counter',j
            
				score, score_list = compute_score(true_events, test_events)
				#print 'score on return',score                
				if score > 0:
					#print 'here is the score for guid:',processing,score            
					score_matrix[i, j] = score
					score_detail[(i,j)] = {"score": score, "detail": score_list, "pair": [test_event]}                    
                
					#print 'here is the current score detail',score_detail[(i,j)]  
				#else:
					#print 'No Score for this guid',processing                                
		#valerie = max(score_detail, key=lambda x:x['score'])
		#print score_detail
		#max_scoreid = max(score_detail, key=lambda i: score_detail[i])
		#print 'max_scorekey',max_scoreid
		#print 'highest scored record',score_detail[max_scoreid]       
		cost_matrix = 10000 - score_matrix
		#print 'here'    
		m = Munkres()

		#print 'here is cost_matrix',cost_matrix    
    
		indexes = m.compute(cost_matrix.tolist())
		#print 'here are my indexes',indexes
		total_score = sum(score_matrix[zip(*indexes)])
		event_pairs = []
		#going to track total score 
		#print 'the total score field is',total_score
		vendor_score += total_score        
		#print 'The total score for guid %s is: %s'%(processing,vendor_score)        
		unmatched_events = []
		spurious_events = []
		#matched_true_ids, matched_test_ids = zip(*indexes)
		#print matched_true_ids
	return vendor_score     

# --------------------just a line break for better reading------------------
# start of processing
# let's define our data to test
#****Processing notes, we need to now address multiple clusters. We are processing for a single cluster
#****we need to capture the cluster id, flatten and then score.

#vendor_data = '/Users/Pete/Customers/scoring_gt16_2.csv'
#vendor_data='/Users/Pete/Customers/scoring_gt16_subset1.csv'
#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gt6.score.csv'
#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gt16.score.csv'
#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gt10.score.csv'
#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gt14.score.csv'
#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/GT7.scoring.csv'

#vendor_processing = raw_input('Which vendor are we processing? ')


#vendor_processing = 'Tamr'

vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gfcluster1.score.csv'

#vendor_data = '/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/tamr.cluster2tet.csv'
with open(vendor_data, 'rU') as f:  #opens PW file
    reader = csv.reader(f)
    dsvendor = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists

#gt_data = '/Users/Pete/Customers/scoring_groundtruthcombined_2.csv'
#gt_data = '/Users/Pete/Customers/scoring_groundtruthcombined_subset1.csv'
#gt_data = '/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/groundtruthcombined.score.csv'
gt_data = '/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gfgroundtruthtest1.score.csv'
#gt_data = '/Users/Pete/Customers/gt.matchtest1'

with open(gt_data, 'rU') as f:  #opens PW file
    reader = csv.reader(f)
    dsgold = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists
    
unmatched = 0
matched = 0
spurious = 0
total_score = 0. 
total_actor1_score = 0.
total_actor2_score = 0.
total_location_score = 0.
total_action_score = 0.
total_date_score = 0.

# PJD this comment is from original source -->This seems silly, but seems to be necessary - HPCC lists passed in have no length
# PJD don't think we need this 
'''
dsgold = []
for item in ds_gold:
	dsgold.append(item)
dsvendor = []
for item in ds_vendor:
	dsvendor.append(item)
'''

#flatten out of ground truth data, only do this once
gtactor1,gtactor2,gtactor1mentions,gtactor2mentions,gtfullgeonames,gtcameocodes,gtguids = reduce_gtdata(dsgold)

clusters_to_process = []

#get all the clusters
for counter,clusters in enumerate(dsvendor):
    clusters_to_process.append(dsvendor[counter][8])
    
clusters = list(set(clusters_to_process))    
print 'our unique cluster ids',clusters

#we will need to check for cluster id and iterate and score based on each cluster id, lets move gtdata higher up so we only need to process it once.
#what if we filter based on cluster id and process each individually

#now loop through and process each cluster
for count,elements in enumerate(clusters):
    #print 'elements',elements
       
    # this doesn't error out but not what we want --> dsvendor_clusters = [item for item in dsvendor if filter_on == elements]
    dsvendor_clusters = filter(lambda dsvendor: dsvendor[8] in elements,dsvendor)
    #dsvendor_clusters = filter(lambda dsvendor: dsvendor[count][8] in elements, dsvendor)
    #print 'iterating through clusters to process',dsvendor_clusters
    

    vendoractor1,vendoractor2,vendoractor1mentions,vendoractor2mentions,vendorfullgeonames,vendorcameocodes,vendorguids,vendorid,clusterid = reduce_vendordata(dsvendor_clusters)

    #print '1.cluster being process',clusterid
    #print '2.vendorguids returned from flatten',vendorguids
    #print '3.gtguids',gtguids

    #get our list of vendor items to score only if they exist in groundtruth
    '''
    vendora1list = [x for x in vendoractor1 if x in gtactor1]
    vendora2list = [x for x in vendoractor2 if x in gtactor2]
    vendora1mlist =[x for x in vendoractor1mentions if x in gtactor1mentions]
    vendora2mlist =[x for x in vendoractor2mentions if x in gtactor2mentions]
    vendorgeolist =[x for x in vendorfullgeonames if x in gtfullgeonames]
    vendoreventlist = [x for x in vendorcameocodes if x in gtcameocodes]
    vendorguidlist = [x for x in vendorguids if x in gtguids]

    '''
    #print '4.guids to process',vendorguidlist

    if not vendoractor1:
        vendoractor1.append('EMPTY')
    if not vendoractor2:
        vendoractor2.append('EMPTY')
    if not vendoractor1mentions:
        vendoractor1mentions.append('EMPTY')
    if not vendoractor2mentions:
        vendoractor2mentions.append('EMPTY')
    if not vendorfullgeonames:
        vendorfullgeonames.append('EMPTY')
    if not vendorcameocodes:
        vendorcameocodes.append('EMPTY')
    if not vendorguids:
        vendorguids.append('EMPTY')

#    if not vendoreventlist:
#        vendoreventlist.append('EMPTY')
#    if not vendorguidlist:
#        vendorguidlist.append('EMPTY')

    #print 'a1 %s,a2 %s,a1m %s,a2m %s,geo %s,cameo %s,guids %s'%(vendora1list,vendora2list,vendora1mlist,vendora2mlist,vendorgeolist,vendoreventlist,vendorguidlist)
    
    #build our scoring loop

    #passing in variables cleaned by ground truth
    #vendor_matrix = list(itertools.product(vendorids,vendora1list,vendora2list,vendora1mlist,vendora2mlist,vendorgeolist,vendoreventlist,vendorguidlist))

    #passing in variables not parsed down from ground truth
    vendor_matrix = list(itertools.product(vendorids,vendoractor1,vendoractor2,vendoractor1mentions,vendoractor2mentions,vendorfullgeonames,vendorcameocodes,vendorguids))

    
    #print 'my length is',len(vendor_matrix)
    #print vendor_matrix


    total_score = global_match(dsgold, vendor_matrix)

    print 'The total score for vendor %s is: %s'%(vendor_processing,total_score) 



our unique cluster ids ['AAA', 'BBB']
The total score for vendor tamr is: 18.0
The total score for vendor tamr is: 13.0
